<img src="https://ucfai.org/groups/core/sp20/02-12-meeting03/meeting03/banner.png">

<div class="col-12">
    <span class="btn btn-success btn-block">
        Meeting in-person? Have you signed in?
    </span>
</div>

<div class="col-12">
    <h1>How We Can Give Our Computers Eyes and Ears </h1>
    <hr>
</div>

<div style="line-height: 2em;">
    <p>by: 
        <strong> Daniel Silva</strong>
        (<a href="https://github.com/danielzgsilva">@danielzgsilva</a>)<br> &emsp;
     on 2019-10-16</p>
</div>

# Convolutional Neural Networks and Transfer Learning Presentation

By the end of todays lesson you will:
-  Know which kinds of problems can be solved using CNNs
-  Understand what a CNN learns and how it learns
-  Have an intution behind how a CNN extracts features from data, and uses these features to generate predictions
-  Be able to utilize transfer learning to achieve state of the art accuracy on novel datasets

## What is a Convolutional Neural Network?

CNNs, like the basic neural networks we've been working with, are simply comprised of sequences of linear layers, followed by nonlinearities. There will still be an Input Layer and Output Layer, with intermediate layers known as Hidden Layers. So what makes a CNN any different? <br>

In short, the goal of any CNN is to extract **features** from its input, and use these features to predict the class of its input, based on what the model has learnt about a dataset.

 <img src = "https://drive.google.com/uc?id=1Sw2o_Lo7jWCs2Ek2qdPfuVB1PqkVgd7l">

#### What kinds of problems can be solved using CNNs?

Due to the nature of CNNs they can, and have been, applied to virtually any kind of data that could have features extracted from it <br>Some applications of CNNs include:
-  Audio processing or classification 
-  Natural Lanuage Processing: feature extraction from text
-  Image Classification and Object Detection
    
For today's lesson we will be looking at CNN's most popular application, Computer Vision. This field deals with how computers can be made to gain understanding from images or videos.

Recent advances in Computer Vision have made tasks such as Image Classification and Object Recognition more accessible and accurate than ever before. 

<img src="https://drive.google.com/uc?id=16V6BTGA0zMny3QRNYym2rIc9W5o52CwT">

#### So how are features extracted from data? 

In a typical feed forward NN the linear layers of the model are essentially just matrix multiplications of the entire input matrix by weight matrices.

With CNNs, however, the weights of the model now consist of **filters**, or **kernels**, and the linear layers are known as **Convolutional Layers**. Think of a filter as an (n*n) matrix which just consists of very specific numbers. 

In a Convolutional Layer, these filters are **convoluted** across the input, performing a computatation known as a **Convolution** against the input data. Performing these convolutions against the input data, with very specific filters, is what allows the CNN to extract features from its input.

So basically, the training of a CNN simply involves finding of the best values for each of these filters so that when an input image is passed through the model's layers, specific features are extracted and certain neurons of the last layer are activated so as to predict the correct class.

## Understanding Convolutions


In order to understand the math behind these convolutions we'll visit Victor Powell's page on Setosa.io, which does an excellent job of visualizing the power of convolution.

[Convolution Visualization](http://setosa.io/ev/image-kernels/)
<br> &emsp;&emsp;By Victor Powell

<img src = "https://drive.google.com/uc?id=1aCqVTSeizgP32uoeUQrgMDG6LyQjA2I2">

In short, the above process of taking each (n x n) area in an image, performing an element wise multiplication against a kernel, and adding each of the resulting numbers together to create one output is called a **convolution**

However, in our dog breeds dataset, images are now colored, rather than white and black, meaning each pixel is on an RGB scale. We now say that the image has 3 channels, one for red, green, and blue. Each input image is now comprised of 3 matrices, each representing a color. In PyTorch speak, the input is now a rank 3 tensor (height x width x 3).

<img src = "https://drive.google.com/uc?id=1JQeOW8fR59chlryITJp8UMqztcj7FED_">


Now that our input is 3 channels deep, we don't want to use the same filter for each of the red, green, and blue channels. Imagine, for example, that we are trying to recognize frogs from images. We'd want to activate the green channel much more than blue or red... Makes sense right? Therefore, for RGB images, the initial filters are also 3 channels deep (height x width x 3), to match the input.

<img src = "https://drive.google.com/uc?id=1Nlunme-oWCidedcAUyqJfO50m9xUVIz4">

Even with 3 channels, the concept of sliding and multipling this filter across the input stays the same. However, rather than a single convolution being an element-wise multiplication of 9 things, it's now an element-wise multiplication of 27 things (3 x 3 x 3). Each of these 27 results are still added up into a single number.

<img src = "https://drive.google.com/uc?id=1-z6ng3-crDxSPXX5WeXGWukDnHmI2FMp">

After performing these convolutions across the entire 3 channel input, we're left with an output of only one 1 channel. This is a rank 2 tensor, or 2-d matrix, of what we call **activations**. As we saw earlier, the filter may have activated all of the top edges in the image, for example.

### Layer 1
<img src = "https://drive.google.com/uc?id=19gYZxzqGYykY6rV5yCXWc9Mg-D8ZyNL1">

To extract additional features from the image, we simply apply more filters to the image, each with the goal of **activating** a different feature. Every applied filter results in another 2-d activation matrix, and we stack these along the 3rd dimension to create a multi channel output (16 channels in this case), or rank 3 tensor. 

Each channel in the output now represents a different feature from the image. Each pixel in one channel might represent much left edge was present in a group of pixels, while another channel might represent how much blue to green gradient was present in each group of pixels.

However, thus far all we have is this rank 3 tensor with channels representing simple things such as where the lines or gradients are in the image... So how can our model begin to learn and activate more complex features?

### Layer 2
<img src = "https://drive.google.com/uc?id=1bGHEVIl2y1sDa6PdrRRdaqo6QU9tkgUf">

At this point we feed this 3 dimensional output of the first layer into the next convolutional layer. Again we choose the number of filters to apply, or essentially the number of features we want extracted, and that creates another rank 3 tensor (again height by width by whatever we chose).

The only difference now is that these kernels can't only have 3 channels anymore can they? Their number of channels need to match the output of the previous layer. 

What this does is allow this set of kernels to perform convolutions on all of the previously activated features, simultaneously.. In doing so, the model is able to activate more and more complex features as the image is passed into deeper layers.

To provide some intuition, let's say the first layer uses 2 filters to extract top and right edges, and stacks these activation matrices into a 2 channel output. The next layer could then apply a 2 channel kernel to this output, correct? So, because layer 2's kernel looks at these channels together, one representing top edges and one representing right edges, we could train this kernel to activate only **top right edges** from the 2 channel input.

## Visualizing the Filters and Features

Let's take a look at what exactly these filters are learning. These visualizations come from an awesome paper written by Matt Zeiler, which is all about visualizing the layers of a Convolutional Neural Network. The specific CNN that the paper illustrates is ResNet34, a popular model for Computer Vision which achieves groundbreaking accuracy on the ImageNet dataset. 

ImageNet is a standard in community for benchmarking networks. It contains millions of images of several thousand different classes, things such as cats, cars, and strawberries. The model actually has 34 layers, and ResNet50 has 50 layers, but let's just start with layer 1. There's a few dozen filters in the first layer so we'll just look at a random 9.

<img src = "https://drive.google.com/uc?id=13vCidyvn5hJVlC2I0pC_NZNUSqaI6EFM">

Remember that these kernels operate on groups of pixels adjacent to each other. So it looks like a few of these try to find groups of pixels that form a diagonal line, while others would find gradients that go from yellow to blue, and so forth. They are simple filters that find lines or gradients, etc.

<img src = "https://drive.google.com/uc?id=1F1saYAtvtAehbEZVZvdf8AIYRH1i7p-a">

Layer 2 then takes the results of those filters and perform convolutions on all these simple features stacked together. The left side of the above image are the filters themselves, and the right side shows images that strongly activated these filters. As you can see, the model has now learned slightly more complex filters. There are some that look for top left corners, or right-hand curves,  circles, etc. So, while in layer 1 we had things that could extract just single lines, for example, layer 2's filters can now find bits of images that have two lines joined up or one line repeated, etc.

<img src = "https://drive.google.com/uc?id=13h3lOQAn6DhwWqlakD-W1QvWRZneB4RD">

At this point the model seems to be able to find two dimensional objects, bits of text, geometric patterns, etc. Overall these filters seem to extract repeating patterns, which put together begin to resemble every day objects/images. So again, layer 3 was able to take all the stuff from layer 2 and combine them together to find even richer features.

<img src = "https://drive.google.com/uc?id=13SXsVOk6Ih9VrBh9kOG125h1Eqrgm6yc">

By layer 4 we've got filters that can find generic dog faces, or bird legs, bodies of water, and other bits of objects.
And by layer 5, we've got something that can find the eyeballs of bird and lizards, or faces of particular breeds of dogs, and so forth. So now you see, with this approach of stacking increasingly complex features, how with only a few layers the model can extract these rich features from am image.

In practice this is how it kind of works. The earlier layers learn to activate very simple shapes or gradients, and feed these to following layers to extract richer and richer features. And by the time the input has reached the deepest layers, the model has extracted all the key features needed to classify a 3 channel matrix of pixels into a real world object.

#### Summary of Convolutional Layers

In summary, most **image** models start with a **3 channel input** (height x width x 3), which is passed through a chosen number of **convolutional kernels** (or filters). This gives us an output of height by width by how ever many kernels we chose, where each channel (or depth dimension) represents an activated feature. This process makes up the bulk of one **convolutional layer**

The output of this layer then becomes the input for the next convolutional layer, and the same process is applied. Remember, now the number of channels in each kernel needs to match the number of channels outputted from the previous layer. This allows each set of kernels to examine all of the features extracted thus far, combined. This process of having filters look at all of the previously extracted features at once allows the model to extract increasingly richer features from the input as the image is passed through the model's layers.

## Additional Details Regarding Convolutions

#### Stride

Stride represents the number of pixels the kernels skip over after each convolution is performed. See the visualizations below for a demonstration. A larger stride helps to reduce the spatial dimensions of the layer's output and reduce the computation required, while a smaller stride captures the finer details and relationships between nearby pixels in an input.
 
##### &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Stride = 1
<img src="https://drive.google.com/uc?id=11yIZzO8IjEhx7QyVAD2n8ykGmV_DGrfS"> 
##### &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Stride = 2
<img src="https://drive.google.com/uc?id=1tVHxKi3XvxaGZKh804-4LVgrvX2OCHgQ"> 

#### Padding

Sometimes, the convolutional filter we choose does not fit perfectly into the input matrix after 'sliding' it across the input. This requires us to pad the input with additional rows or columns. **Padding** can be accomplished using a few different techniques:

*   Zero Padding - pads the input with zeros
*   Valid Padding - drops the part of the input where the filter did not fit.
*   Reflective Padding - pads outer edges with their reflections



We'll illustate both the padding and stride parameters in further detail in the following section.

## End to End Convolutional Neural Networks

Now that we have a base understanding of how a CNN's convolutional layers function to extract features, we can go over a few other common layers that make up a typical CNN. By the end of this section, you'll have an idea of what an end to end CNN architecture is comprised of and the purpose behind each component. We'll take a deeper look at each of these in a second when we build a full fledged CNN in Excel, so don't get discouraged if something doesn't make sense at first.

#### Rectified Linear Unit (ReLU)

Remember, a convolution, at heart, is nothing more than a linear mapping.  So, like in other Neural Networks we've studied, every Convolutional Layer (the linear layer) must be followed by a nonlinearity. This is what allows our model to approximate relationships behind data, as described in the [Universal Approximation Theorem](https://en.wikipedia.org/wiki/Universal_approximation_theorem). 

The most common nonlinearity, or **Activation Function**, used in CNN architectures is what's known as a **Rectified Linear Unit**, frequently referred to as a **ReLU**. This sounds fancy, but really a ReLU is  simply the function **output =  Max(0, input)**. In other words, applying a ReLU simply involves replacing every negative value in a Conv Layer's output with a 0. In practice, a ReLU will follow after every convolutional layer in a CNN.

### Regularization Techniques

#### Pooling Layers

Another staple in nearly every CNN architecture are a few **Pooling** layers. Pooling uses a technique known as **Down Sampling**, to reduce the number of parameters in your model, while allowing it to better generalize images across different orientations and scales. This is also known as avoiding **overfitting**. In order to do this a pooling layer uses a sliding technique, similar to our conv. layers, but now applying a statistical function over the contents of its window. The pooling windows are also non-overlapping, allowing this technique to greatly reduce the size of its input and, naturally, decrease the number of parameters in the model.

There are many kinds of pooling layers, each with their own advantages and use cases:

-  Max Pooling
-  Average Pooling
-  Global Poolings
-  Etc.

The diagram below shows an example of max pooling. We'll speak further on Global Pooling, later in today's meeting.

 <img src = "https://drive.google.com/uc?id=1cH0LHtXDd4FJnT3qZQSdNxai7MIc2ra0">


#### Dropout Layers

Another powerful and very popular regularization technique is **Dropout**. Like the name suggests, a dropout layer will randomly drop a chosen percentage of activations by setting them to 0. This means that each batch, at each epoch, will have a random set of activations thrown out, at every drop out layer... 

As you can imagine, this introduces a great deal of variability and forces the model to never memorize any particular arrangement of activations, but to learn to generalize when looking at new, but similar, data. It's important to note that dropout is only used during training. At test time we want to use all the activations possible to achieve the highest accuracy.


#### Batch Normalization

In the past few years of advancement in the field of computer vision, batch norm has stood out as one of the most important techniques for making the training of these large CNNs more accessible to the general public. More specifically, it vastly reduces the number of epochs and training time needed to accurately train a model.

Batch Normalization standardizes the input into a layer for each mini-batch by scaling and shifting each activation, so that all the activations in the layer have a mean of 0 and a variance of 1. This normalization stabilizes the learning process and allows you to increase the learning rate of your model. This stabilization effect is visualized in the image below. 

Explaining exactly why normalizing our inputs has this effect can get quite complicated, but let's think about it for a second. These models are trained through **backpropagation**, meaning, after each mini-batch we calculate our loss, take the gradient, and adjust our weights accordingly so that the next mini-batch has a lower loss with this updated set of weights. Now, because we're constantly updating our weights, this means that the distribution of our inputs is constantly changing after each mini-batch. In short, this is what causes this rough loss landscape that we see below. By normalizing our inputs for each mini-batch we can minimize this effect and greatly smooth out the loss landscape. Now that our loss is much more stable throughout training we can increase the learning rate without much side effect, hence reducing the training time of our model.

<img src = "https://drive.google.com/uc?id=1GPSUxmQziaSReCGWSp7dPg1jo5o9aJzj">

### Classification Layers

So, ultimately, after passing an input through any combination of these layers we've talked about, you end up with a multi channel output of activation matrices, where each channel represents a different high level feature of the original input. This concludes the **feature extraction** portion of a CNN.

Now, the goal of any CNN is to take all of these calculated feature maps, and use them to predict the likelihood that the original input belongs to each class we trained the model on. This is the goal of the **classification** portion of a CNN, the only question remaining is how to accomplish this.

<img src = "https://drive.google.com/uc?id=1mm2_dS5KW9s9v4NPzTRT6l8V2_gs0pt9">

#### Global Average Pooling

In order to apply a fully connected layer to generate our output vector, we need to turn this 3 dimensional output of stacked feature maps into a rank one tensor. One method of doing this is to take the average of each one of these feature maps, which results in a vector of 512 values. Intuitively, each of these values tell us how strongly a feature was present in the original image. Things like whether the original image had brown eyes, or pointy ears, or a fluffy tail, etc.
<img src = "https://drive.google.com/uc?id=1uOpEyVDg30_EdJdvLxv4-NgV3Js3q2zi">

#### Flattening Layer

Another option is to simply flatten the rank 3 tensor of activations into a rank one tensor. In this case, it'd result in a 61,952 length vector (11 x 11 x 512). This is useful because we get to keep every single activation from each feature map, however as you can imagine, it requires significantly more memory and computation power. Because of this, most modern architectures seem to opt for the Global Average Pooling method.

#### Fully Connected Layer (Output Layer)

Regardless of the method you choose, once you're left with a rank one tensor you can now apply a standard fully connected layer (like the ones we saw in our first few neural net meetings). Remember that a fully connected layer is nothing more than a matrix multiplication by a weight matrix, which can be learned. This fully connected layer aims to take our vector of activations and turn it into a vector of length equal to the number of classes we're trying to predict (in this case 37).

From here, the last thing we need to do is apply a final activation function to turn each of these values into a percentage. By far the 2 most popular functions for this are the Softmax and Sigmoid functions, which are both only ever used in the final layer of a CNN.

<img src = "https://drive.google.com/uc?id=1BzI4qohtkhWiKFRTNCrWHFn8XU8cxzrA">

#### Softmax

Softmax is just another non-linearity like ReLU, however, its behavior makes it perfect for use in the final layer of a model predicting a single class for an input. This is because it can take in the output of our fully connected layer and create a probability distribution. In other words, each output value is non-negative and adds to 1. We'll look at an example of this in excel to gain a better understanding in just a second.

<img src = "https://drive.google.com/uc?id=1R6qK3TxhiV_TL1UZr7Njfn1fwXs3hwUc">

#### Sigmoid

Sigmoid, on the other hand, is commonly used for multi-label classification problems. The function's output will always be between 0 and 1, again, making it suitable to generate probablities. Unlike softmax, however, the values within the output vector will not add to one, allowing the input to have high probabilities for multiple classes at once.

<img src = "https://drive.google.com/uc?id=1y30iXT_kwX7zfwpywJtfHT1dF5Ba9ZNb">

#### &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; End to End Model

<img src = "https://drive.google.com/uc?id=1Sw2o_Lo7jWCs2Ek2qdPfuVB1PqkVgd7l">